## Vehicle Class

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime
import glob

def extract_metadata(title_text, file_path):
    """
    Extract RTO name, code and state from the title text
    
    Parameters:
    title_text (str): The title text containing RTO information
    file_path (str): Path to the Excel file, used for fallback extraction
    
    Returns:
    tuple: (rto_name, rto_code, state_name)
    """
    # Regular expression to match the pattern: "of RTO_NAME - CODE , STATE_NAME (YEAR)"
    # Make the pattern more flexible for variations in spacing and format
    pattern = r"of\s+(.+?)\s+-\s+([A-Z0-9]+)\s*[,\s]+\s*(.+?)\s*\(\d{4}\)"
    match = re.search(pattern, title_text)
    
    if match:
        rto_name = match.group(1).strip()
        rto_code = match.group(2).strip()
        state_name = match.group(3).strip()
        return rto_name, rto_code, state_name
    else:
        # Fall back to another pattern if the first one doesn't match
        # Extract from filename for files like "BARPETA - AS15.xlsx"
        filename_pattern = r"([A-Za-z\s]+)\s*-\s*([A-Z0-9]+)"
        match = re.search(filename_pattern, os.path.basename(file_path))
        if match:
            rto_name = match.group(1).strip()
            rto_code = match.group(2).strip()
            # Try to extract state from path
            state_name = os.path.basename(os.path.dirname(file_path))
            return rto_name, rto_code, state_name
        
        return None, None, None

def process_excel_file(file_path):
    """
    Process a single Excel file and extract relevant data
    """
    try:
        # First, try to read the Excel file with default settings
        df = pd.read_excel(file_path, header=None)
        
        # Look for title text in first few rows
        title_text = None
        for i in range(min(5, len(df))):
            if pd.notna(df.iloc[i, 0]) and "Vehicle Class Month Wise Data" in str(df.iloc[i, 0]):
                title_text = str(df.iloc[i, 0])
                break
        
        if title_text is None:
            title_text = f"Vehicle Class Month Wise Data of {os.path.basename(file_path).split('.')[0]}"
        
        # Extract metadata with correct number of arguments
        rto_name, rto_code, state_name = extract_metadata(title_text, file_path)
        if not rto_name:
            rto_name = os.path.basename(file_path).split('.')[0]
        if not rto_code:
            rto_code = "N/A"
        if not state_name:
            state_name = os.path.basename(os.path.dirname(file_path))
        
        # Look for month columns more flexibly
        month_names = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 
                       'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
                     
        # Find the row with month names by searching more broadly
        header_row_idx = None
        month_cols = []
        month_indices = []
        
        for i in range(len(df)):
            month_found = False
            for j in range(len(df.columns)):
                cell_value = str(df.iloc[i, j]).upper() if pd.notna(df.iloc[i, j]) else ""
                if any(month in cell_value for month in month_names):
                    month_found = True
                    break
            
            if month_found:
                header_row_idx = i
                break
        
        if header_row_idx is None:
            print(f"Could not find header row with months in {file_path}")
            return None
            
        # Find month columns in the header row
        for j in range(len(df.columns)):
            cell_value = str(df.iloc[header_row_idx, j]).upper() if pd.notna(df.iloc[header_row_idx, j]) else ""
            for month in month_names:
                if month == cell_value or cell_value.endswith(f" {month}"):
                    month_cols.append(month)
                    month_indices.append(j)
                    break
        
        if not month_cols:
            # If months not found directly, check if they're in a combined cell
            for j in range(len(df.columns)):
                cell_value = str(df.iloc[header_row_idx, j]).upper() if pd.notna(df.iloc[header_row_idx, j]) else ""
                found_months = []
                for month in month_names:
                    if month in cell_value:
                        found_months.append(month)
                
                if len(found_months) > 0:
                    # If multiple months in one cell, assume they're in consecutive columns
                    for idx, month in enumerate(found_months):
                        month_cols.append(month)
                        month_indices.append(j + idx)
        
        if not month_cols:
            print(f"Could not find month columns in {file_path}")
            return None
        
        # Find vehicle class column - typically the second column after S No
        vehicle_class_col = None
        for j in range(len(df.columns)):
            if pd.notna(df.iloc[header_row_idx, j]) and "VEHICLE CLASS" in str(df.iloc[header_row_idx, j]).upper():
                vehicle_class_col = j
                break
        
        if vehicle_class_col is None:
            # If not found, assume it's column 1 (second column)
            vehicle_class_col = 1
        
        # Read data rows (skip header and start from next row)
        data_start_idx = header_row_idx + 1
        data_rows = []
        
        for i in range(data_start_idx, len(df)):
            row = df.iloc[i]
            
            # Skip empty rows or rows without vehicle class
            if pd.isna(row[vehicle_class_col]) or str(row[vehicle_class_col]).strip() == "":
                continue
            
            vehicle_class = str(row[vehicle_class_col]).strip()
            
            # Skip rows that don't look like data rows
            if vehicle_class.isdigit() or vehicle_class.upper() == "TOTAL":
                continue
            
            # Extract monthly data
            month_data = {}
            for month, col_idx in zip(month_cols, month_indices):
                if col_idx < len(row):
                    value = row[col_idx]
                    if pd.isna(value):
                        value = 0
                    month_data[month] = value
                else:
                    month_data[month] = 0
            
            # Create a row with all information
            data_row = {
                'RTO Name': rto_name,
                'RTO Code': rto_code,
                'State': state_name,
                'Vehicle Class': vehicle_class,
                **month_data
            }
            data_rows.append(data_row)
        
        return data_rows
        
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

def main():
    # Get all Excel files in the folder and subfolders
    folder_path = r"F:\Flipcarbon\Ajax\Vehicle Class" # Change this to your folder path
    excel_files = glob.glob(os.path.join(folder_path, "**", "*.xlsx"), recursive=True)
    
    if not excel_files:
        print("No Excel files found in the specified folder and subfolders")
        return
    
    # Process each file and collect data
    all_data = []
    for file_path in excel_files:
        print(f"Processing {file_path}...")
        file_data = process_excel_file(file_path)
        if file_data:
            all_data.extend(file_data)
    
    if not all_data:
        print("No data was extracted from the files")
        return
    
    # Create a DataFrame from all data
    result_df = pd.DataFrame(all_data)
    
    # Generate output filename with today's date
    today = datetime.now().strftime("%d-%m-%Y")
    output_filename = f"Consolidated Vehicle Class Wise Data ({today}).xlsx"
    
    # Save to Excel
    result_df.to_excel(output_filename, index=False)
    print(f"Data has been saved to {output_filename}")

if __name__ == "__main__":
    main()

## Maker

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime
import glob

def extract_metadata(title_text, file_path):
    """
    Extract RTO name, code and state from the title text
    
    Parameters:
    title_text (str): The title text containing RTO information
    file_path (str): Path to the Excel file, used for fallback extraction
    
    Returns:
    tuple: (rto_name, rto_code, state_name)
    """
    # Regular expression to match the pattern: "of RTO_NAME - CODE , STATE_NAME (YEAR)"
    # Make the pattern more flexible for variations in spacing and format
    pattern = r"of\s+(.+?)\s+-\s+([A-Z0-9]+)\s*[,\s]+\s*(.+?)\s*\(\d{4}\)"
    match = re.search(pattern, title_text)
    
    if match:
        rto_name = match.group(1).strip()
        rto_code = match.group(2).strip()
        state_name = match.group(3).strip()
        return rto_name, rto_code, state_name
    else:
        # Fall back to another pattern if the first one doesn't match
        # Extract from filename for files like "BARPETA - AS15.xlsx"
        filename_pattern = r"([A-Za-z\s]+)\s*-\s*([A-Z0-9]+)"
        match = re.search(filename_pattern, os.path.basename(file_path))
        if match:
            rto_name = match.group(1).strip()
            rto_code = match.group(2).strip()
            # Try to extract state from path
            state_name = os.path.basename(os.path.dirname(file_path))
            return rto_name, rto_code, state_name
        
        return None, None, None

def process_excel_file(file_path):
    """
    Process a single Excel file and extract relevant data
    """
    try:
        # First, try to read the Excel file with default settings
        df = pd.read_excel(file_path, header=None)
        
        # Look for title text in first few rows
        title_text = None
        for i in range(min(5, len(df))):
            if pd.notna(df.iloc[i, 0]) and "Maker Month Wise Data" in str(df.iloc[i, 0]):
                title_text = str(df.iloc[i, 0])
                break
        
        if title_text is None:
            title_text = f"Maker Month Wise Data of {os.path.basename(file_path).split('.')[0]}"
        
        # Extract metadata with correct number of arguments
        rto_name, rto_code, state_name = extract_metadata(title_text, file_path)
        if not rto_name:
            rto_name = os.path.basename(file_path).split('.')[0]
        if not rto_code:
            rto_code = "N/A"
        if not state_name:
            state_name = os.path.basename(os.path.dirname(file_path))
        
        # Look for month columns more flexibly
        month_names = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 
                       'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
                     
        # Find the row with month names by searching more broadly
        header_row_idx = None
        month_cols = []
        month_indices = []
        
        for i in range(len(df)):
            month_found = False
            for j in range(len(df.columns)):
                cell_value = str(df.iloc[i, j]).upper() if pd.notna(df.iloc[i, j]) else ""
                if any(month in cell_value for month in month_names):
                    month_found = True
                    break
            
            if month_found:
                header_row_idx = i
                break
        
        if header_row_idx is None:
            print(f"Could not find header row with months in {file_path}")
            return None
            
        # Find month columns in the header row
        for j in range(len(df.columns)):
            cell_value = str(df.iloc[header_row_idx, j]).upper() if pd.notna(df.iloc[header_row_idx, j]) else ""
            for month in month_names:
                if month == cell_value or cell_value.endswith(f" {month}"):
                    month_cols.append(month)
                    month_indices.append(j)
                    break
        
        if not month_cols:
            # If months not found directly, check if they're in a combined cell
            for j in range(len(df.columns)):
                cell_value = str(df.iloc[header_row_idx, j]).upper() if pd.notna(df.iloc[header_row_idx, j]) else ""
                found_months = []
                for month in month_names:
                    if month in cell_value:
                        found_months.append(month)
                
                if len(found_months) > 0:
                    # If multiple months in one cell, assume they're in consecutive columns
                    for idx, month in enumerate(found_months):
                        month_cols.append(month)
                        month_indices.append(j + idx)
        
        if not month_cols:
            print(f"Could not find month columns in {file_path}")
            return None
        
        # Find vehicle class column - typically the second column after S No
        Maker_col = None
        for j in range(len(df.columns)):
            if pd.notna(df.iloc[header_row_idx, j]) and "MAKER" in str(df.iloc[header_row_idx, j]).upper():
                Maker_col = j
                break
        
        if Maker_col is None:
            # If not found, assume it's column 1 (second column)
            Maker_col = 1
        
        # Read data rows (skip header and start from next row)
        data_start_idx = header_row_idx + 1
        data_rows = []
        
        for i in range(data_start_idx, len(df)):
            row = df.iloc[i]
            
            # Skip empty rows or rows without vehicle class
            if pd.isna(row[Maker_col]) or str(row[Maker_col]).strip() == "":
                continue
            
            Maker = str(row[Maker_col]).strip()
            
            # Skip rows that don't look like data rows
            if Maker.isdigit() or Maker.upper() == "TOTAL":
                continue
            
            # Extract monthly data
            month_data = {}
            for month, col_idx in zip(month_cols, month_indices):
                if col_idx < len(row):
                    value = row[col_idx]
                    if pd.isna(value):
                        value = 0
                    month_data[month] = value
                else:
                    month_data[month] = 0
            
            # Create a row with all information
            data_row = {
                'RTO Name': rto_name,
                'RTO Code': rto_code,
                'State': state_name,
                'Maker': Maker,
                **month_data
            }
            data_rows.append(data_row)
        
        return data_rows
        
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

def main():
    # Get all Excel files in the folder and subfolders
    folder_path = r"F:\Flipcarbon\Ajax\Maker" # Change this to your folder path
    excel_files = glob.glob(os.path.join(folder_path, "**", "*.xlsx"), recursive=True)
    
    if not excel_files:
        print("No Excel files found in the specified folder and subfolders")
        return
    
    # Process each file and collect data
    all_data = []
    for file_path in excel_files:
        print(f"Processing {file_path}...")
        file_data = process_excel_file(file_path)
        if file_data:
            all_data.extend(file_data)
    
    if not all_data:
        print("No data was extracted from the files")
        return
    
    # Create a DataFrame from all data
    result_df = pd.DataFrame(all_data)
    
    # Generate output filename with today's date
    today = datetime.now().strftime("%d-%m-%Y")
    output_filename = f"Consolidated Maker Wise Data ({today}).xlsx"
    
    # Save to Excel
    result_df.to_excel(output_filename, index=False)
    print(f"Data has been saved to {output_filename}")

if __name__ == "__main__":
    main()

## Vehicle Category

In [1]:
import os
import pandas as pd
import re
from datetime import datetime
import glob

def extract_metadata(title_text, file_path):
    """
    Extract RTO name, code and state from the title text
    
    Parameters:
    title_text (str): The title text containing RTO information
    file_path (str): Path to the Excel file, used for fallback extraction
    
    Returns:
    tuple: (rto_name, rto_code, state_name)
    """
    # Regular expression to match the pattern: "of RTO_NAME - CODE , STATE_NAME (YEAR)"
    # Make the pattern more flexible for variations in spacing and format
    pattern = r"of\s+(.+?)\s+-\s+([A-Z0-9]+)\s*[,\s]+\s*(.+?)\s*\(\d{4}\)"
    match = re.search(pattern, title_text)
    
    if match:
        rto_name = match.group(1).strip()
        rto_code = match.group(2).strip()
        state_name = match.group(3).strip()
        return rto_name, rto_code, state_name
    else:
        # Fall back to another pattern if the first one doesn't match
        # Extract from filename for files like "BARPETA - AS15.xlsx"
        filename_pattern = r"([A-Za-z\s]+)\s*-\s*([A-Z0-9]+)"
        match = re.search(filename_pattern, os.path.basename(file_path))
        if match:
            rto_name = match.group(1).strip()
            rto_code = match.group(2).strip()
            # Try to extract state from path
            state_name = os.path.basename(os.path.dirname(file_path))
            return rto_name, rto_code, state_name
        
        return None, None, None

def process_excel_file(file_path):
    """
    Process a single Excel file and extract relevant data
    """
    try:
        # First, try to read the Excel file with default settings
        df = pd.read_excel(file_path, header=None)
        
        # Look for title text in first few rows
        title_text = None
        for i in range(min(5, len(df))):
            if pd.notna(df.iloc[i, 0]) and "Vehicle Category Month Wise Data" in str(df.iloc[i, 0]):
                title_text = str(df.iloc[i, 0])
                break
        
        if title_text is None:
            title_text = f"Vehicle Category Month Wise Data of {os.path.basename(file_path).split('.')[0]}"
        
        # Extract metadata with correct number of arguments
        rto_name, rto_code, state_name = extract_metadata(title_text, file_path)
        if not rto_name:
            rto_name = os.path.basename(file_path).split('.')[0]
        if not rto_code:
            rto_code = "N/A"
        if not state_name:
            state_name = os.path.basename(os.path.dirname(file_path))
        
        # Look for month columns more flexibly
        month_names = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 
                       'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
                     
        # Find the row with month names by searching more broadly
        header_row_idx = None
        month_cols = []
        month_indices = []
        
        for i in range(len(df)):
            month_found = False
            for j in range(len(df.columns)):
                cell_value = str(df.iloc[i, j]).upper() if pd.notna(df.iloc[i, j]) else ""
                if any(month in cell_value for month in month_names):
                    month_found = True
                    break
            
            if month_found:
                header_row_idx = i
                break
        
        if header_row_idx is None:
            print(f"Could not find header row with months in {file_path}")
            return None
            
        # Find month columns in the header row
        for j in range(len(df.columns)):
            cell_value = str(df.iloc[header_row_idx, j]).upper() if pd.notna(df.iloc[header_row_idx, j]) else ""
            for month in month_names:
                if month == cell_value or cell_value.endswith(f" {month}"):
                    month_cols.append(month)
                    month_indices.append(j)
                    break
        
        if not month_cols:
            # If months not found directly, check if they're in a combined cell
            for j in range(len(df.columns)):
                cell_value = str(df.iloc[header_row_idx, j]).upper() if pd.notna(df.iloc[header_row_idx, j]) else ""
                found_months = []
                for month in month_names:
                    if month in cell_value:
                        found_months.append(month)
                
                if len(found_months) > 0:
                    # If multiple months in one cell, assume they're in consecutive columns
                    for idx, month in enumerate(found_months):
                        month_cols.append(month)
                        month_indices.append(j + idx)
        
        if not month_cols:
            print(f"Could not find month columns in {file_path}")
            return None
        
        # Find vehicle class column - typically the second column after S No
        vehicle_category_col = None
        for j in range(len(df.columns)):
            if pd.notna(df.iloc[header_row_idx, j]) and "VEHICLE CATEGORY" in str(df.iloc[header_row_idx, j]).upper():
                vehicle_category_col = j
                break
        
        if vehicle_category_col is None:
            # If not found, assume it's column 1 (second column)
            vehicle_category_col = 1
        
        # Read data rows (skip header and start from next row)
        data_start_idx = header_row_idx + 1
        data_rows = []
        
        for i in range(data_start_idx, len(df)):
            row = df.iloc[i]
            
            # Skip empty rows or rows without vehicle class
            if pd.isna(row[vehicle_category_col]) or str(row[vehicle_category_col]).strip() == "":
                continue
            
            vehicle_category = str(row[vehicle_category_col]).strip()
            
            # Skip rows that don't look like data rows
            if vehicle_category.isdigit() or vehicle_category.upper() == "TOTAL":
                continue
            
            # Extract monthly data
            month_data = {}
            for month, col_idx in zip(month_cols, month_indices):
                if col_idx < len(row):
                    value = row[col_idx]
                    if pd.isna(value):
                        value = 0
                    month_data[month] = value
                else:
                    month_data[month] = 0
            
            # Create a row with all information
            data_row = {
                'RTO Name': rto_name,
                'RTO Code': rto_code,
                'State': state_name,
                'Vehicle Category': vehicle_category,
                **month_data
            }
            data_rows.append(data_row)
        
        return data_rows
        
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

def main():
    # Get all Excel files in the folder and subfolders
    folder_path = r"C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh" # Change this to your folder path
    excel_files = glob.glob(os.path.join(folder_path, "**", "*.xlsx"), recursive=True)
    
    if not excel_files:
        print("No Excel files found in the specified folder and subfolders")
        return
    
    # Process each file and collect data
    all_data = []
    for file_path in excel_files:
        print(f"Processing {file_path}...")
        file_data = process_excel_file(file_path)
        if file_data:
            all_data.extend(file_data)
    
    if not all_data:
        print("No data was extracted from the files")
        return
    
    # Create a DataFrame from all data
    result_df = pd.DataFrame(all_data)
    
    # Generate output filename with today's date
    today = datetime.now().strftime("%d-%m-%Y")
    output_filename = f"Consolidated Vehicle Category Wise Data ({today}).xlsx"
    
    # Save to Excel
    result_df.to_excel(output_filename, index=False)
    print(f"Data has been saved to {output_filename}")

if __name__ == "__main__":
    main()

Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\ANJAW - AR17.xlsx...


c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's de

Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_1.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_10.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_11.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_12.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_13.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_14.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_15.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_16.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_17.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_18.xlsx...


c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's de

Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_19.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_2.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_20.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_21.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_22.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_23.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_24.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_25.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_26.xlsx...


c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's de

Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_27.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_3.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_4.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_5.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_6.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_7.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_8.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\_9.xlsx...
Processing C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\~$_2.xlsx...
Error processing file C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh\~$_2.xlsx: [Errno 13] Permission denied: 'C:\\Users\\ASUS\\OneDrive\\Documents\\Maker Wise\\Arunachal Pradesh\\~$_2.xlsx'
Data has been saved to Consolidated